In [2]:
import requests
from aws_requests_auth.aws_auth import AWSRequestsAuth
import os
#https://www.elastic.co/guide/en/elasticsearch/reference/current/number.html
#https://github.com/DavidMuller/aws-requests-auth 

#https://www.elastic.co/guide/en/elasticsearch/reference/current/_batch_processing.html




In [20]:
AWS_SEARCH_HOST = os.environ.get("AWS_ES_NCBI_HOST")

HOST_URL = 'https://' + AWS_SEARCH_HOST
AWS_ACCES_KEY = os.environ.get("AWS_KEY_ID")
AWS_ACCESS_SECRET = os.environ.get("AWS_KEY_SECRET")

auth = AWSRequestsAuth(aws_access_key=AWS_ACCES_KEY,
                       aws_secret_access_key=AWS_ACCESS_SECRET,
                       aws_host=AWS_SEARCH_HOST,
                       aws_region='us-east-1',
                       aws_service='es')

response = requests.get(HOST_URL, auth=auth)
print response.content




{
  "name" : "twnJ3PL",
  "cluster_name" : "787588439240:czi-bio-search-alpha",
  "cluster_uuid" : "5PfdcJmmSDmDMKDCd0zsuQ",
  "version" : {
    "number" : "5.1.1",
    "build_hash" : "5395e21",
    "build_date" : "2016-12-15T22:47:19.049Z",
    "build_snapshot" : false,
    "lucene_version" : "6.3.0"
  },
  "tagline" : "You Know, for Search"
}



In [ ]:
# Test post and update functions

In [22]:
response = requests.delete(HOST_URL + '/testindex?pretty', auth=auth)
response.content

response = requests.put(HOST_URL + '/testindex?pretty', auth=auth)
response.content


'{\n  "acknowledged" : true,\n  "shards_acknowledged" : true\n}\n'

In [29]:

response = requests.get(HOST_URL + '/_cat/indices?v', auth=auth)
print response.content


response = requests.get(HOST_URL + '/testindex/_search?q=geo_accession%3AGSM1617399&pretty')
x = response.content
import json
json.loads(x)


health status index     uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   .kibana   PRhwMuurTZ-kZXK1ULxQcg   1   1          1            0      3.1kb          3.1kb
yellow open   testindex BQG4ds8lTo-EC3w1BTeBLw   5   1          9            0    262.6kb        262.6kb



{u'_shards': {u'failed': 0, u'successful': 5, u'total': 5},
 u'hits': {u'hits': [], u'max_score': None, u'total': 0},
 u'timed_out': False,
 u'took': 3}

In [30]:
import GEOparse
gse = GEOparse.get_GEO(filepath="/Users/yf/Downloads/GSE66217_family.soft.gz")
#print gse.metadata
#print gse.gsms # look at samples' .metadata
#print gse.gpls # look at platforms .metadata


def generate_index_objects(gse_file):
    gse = GEOparse.get_GEO(filepath=gse_file)
    # output 
    # {sample_data: , series_data: , platform_data:}
    series_data = gse.metadata
    platform_data = []
    samples = {}
    
    for key, obj in gse.gpls.iteritems():
        obj_hash = obj.metadata
        obj_hash['gpl_key'] = key
        platform_data.append(obj_hash)
        
    for key, obj in gse.gsms.iteritems():
        sample_data = obj.metadata
        samples[key] = {'sample_data':sample_data, 'series_data':series_data, 'platform_data':platform_data}
        
    return samples
    

def generate_index_json(samples):
    output = ''
    for key, sample_obj in samples.iteritems():
        output += '{"index":{"_id":"%s"}}\n' % key
        output += json.dumps(sample_obj) + '\n'
    return output
        

samples = generate_index_objects("/Users/yf/Downloads/GSE66217_family.soft.gz")


#index_data = generate_index_json(samples) 
#response = requests.put(HOST_URL + '/testindex/external/_bulk?pretty', data=index_data, auth=auth)
#print response.content


Parsing /Users/yf/Downloads/GSE66217_family.soft.gz:
 - DATABASE : GeoMiame
 - SERIES : GSE66217
 - PLATFORM : GPL11154
 - SAMPLE : GSM1617398
 - SAMPLE : GSM1617399
 - SAMPLE : GSM1617400
 - SAMPLE : GSM1617401
 - SAMPLE : GSM1617402
 - SAMPLE : GSM1617403
 - SAMPLE : GSM1617404
 - SAMPLE : GSM1617405
 - SAMPLE : GSM1617406
Parsing /Users/yf/Downloads/GSE66217_family.soft.gz:
 - DATABASE : GeoMiame
 - SERIES : GSE66217
 - PLATFORM : GPL11154
 - SAMPLE : GSM1617398
 - SAMPLE : GSM1617399
 - SAMPLE : GSM1617400
 - SAMPLE : GSM1617401
 - SAMPLE : GSM1617402
 - SAMPLE : GSM1617403
 - SAMPLE : GSM1617404
 - SAMPLE : GSM1617405
 - SAMPLE : GSM1617406


In [45]:
x = HOST_URL + '/testindex/_search?q=cortex&pretty'
print x

https://search-czi-bio-search-alpha-6f6daen5vamw4ywwy5uyknspry.us-east-1.es.amazonaws.com/testindex/_search?q=cortex&pretty


In [32]:
samples

{'GSM1617398': {'platform_data': [{'contact_country': ['USA'],
    'contact_name': [',,GEO'],
    'data_row_count': ['0'],
    'distribution': ['virtual'],
    'geo_accession': ['GPL11154'],
    'gpl_key': 'GPL11154',
    'last_update_date': ['Feb 06 2017'],
    'organism': ['Homo sapiens'],
    'status': ['Public on Nov 02 2010'],
    'submission_date': ['Nov 02 2010'],
    'taxid': ['9606'],
    'technology': ['high-throughput sequencing'],
    'title': ['Illumina HiSeq 2000 (Homo sapiens)']}],
  'sample_data': {'channel_count': ['1'],
   'characteristics_ch1': ['gestational age (weeks): 18',
    'facs population: LeX+ Glast+ Prominin-high'],
   'contact_address': ['300 Longwood Ave.'],
   'contact_city': ['Boston'],
   'contact_country': ['USA'],
   'contact_department': ['Genetics and Genomics'],
   'contact_email': ['matthew.johnson@childrens.harvard.edu'],
   'contact_institute': ["Boston Children's Hospital"],
   'contact_name': ['Matthew,Bonser,Johnson'],
   'contact_state': ['

In [9]:
gse.gsms['GSM1617399'].metadata

{'channel_count': ['1'],
 'characteristics_ch1': ['gestational age (weeks): 18',
  'facs population: LeX+ Glast+ Prominin-low'],
 'contact_address': ['300 Longwood Ave.'],
 'contact_city': ['Boston'],
 'contact_country': ['USA'],
 'contact_department': ['Genetics and Genomics'],
 'contact_email': ['matthew.johnson@childrens.harvard.edu'],
 'contact_institute': ["Boston Children's Hospital"],
 'contact_name': ['Matthew,Bonser,Johnson'],
 'contact_state': ['Massachusetts'],
 'contact_zip/postal_code': ['02115'],
 'data_processing': ['base-calling with Illumina CASAVA 1.8.2',
  'alignment to the human reference genome and UCSC Known Genes transcriptome with tophat v2.0.6',
  'gene expression level estimation and differential expression analysis performed with cufflinks v2.0.2',
  'Genome_build: hg19',
  'Supplementary_files_format_and_content: cufflinks gene-level fpkm tracking file format, contains gene-level expression estimates for each condition summarized across the three biological 

In [18]:
gse.gpls['GPL11154'].metadata
print '{"index":{"_id":"%s"}}\n' % "dsfsdf"

print 23434

{"index":{"_id":"dsfsdf"}}

23434
